# Análise exploratório dos dados (EDA).

In [1]:
import pandas as pd
import re

# Importando o arquivo csv de  treino, quantidade linha e colunas e projeção de algumas linha 
train = pd.read_csv('train.csv')
print("Quantidade linhas:%d"%(train.shape[0]))
print("Quantidade linhas: %d"%(train.shape[1]))

train

Quantidade linhas:31962
Quantidade linhas: 3


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


# Pre processamento

In [10]:
def  process_text(dset, column):
    
    # Transforma as frases da coluna em minuscula
    dset[column] = dset[column].str.lower()
    # pega anomalias e substitui por regex
    dset[column] = dset[column].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    
    return dset

train_clean = process_text(train, "tweet")

In [19]:
from sklearn.utils import resample

def balance_class(majority):
    if majority:
        train_majority = train_clean[train_clean.label==0]
        train_minority = train_clean[train_clean.label==1]
    else:
        train_majority = train_clean[train_clean.label==1]
        train_minority = train_clean[train_clean.label==0]
        
    train_majority_down_up_sampled = resample(train_majority, 
                                 replace=True,  
                                 n_samples=len(train_minority),   
                                 random_state=123)
    
    train_down_up_sampled = pd.concat([train_majority_down_up_sampled, train_minority])
    train_down_up_sampled['label'].value_counts()
    return train_down_up_sampled

# Usando balanceamento para maioria e minoria, estabilizando o dataset
train_upsampled = balance_class(True)
train_downsampled = balance_class(False)

# Treinamento 

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

# Pipeline usa o principio do chain of responsability passando o dado por vários passos durante a fase de treinamento
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [21]:
from sklearn.model_selection import train_test_split

# separa o conjunto em treinamento e teste para deixar o modelo mais preciso
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],                                    
                                                    train_upsampled['label'],random_state = 0)


In [24]:

# com os datasets separado é pego os conjuntos de treinamento e treina o modelo
# atráves da estrategia de pipeline
# Com o modelo treinado é hora de verificar se através dos testes ele é preciso
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)

# Atravé do f1 score vemos que ele é 85% preciso
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.852994555353902

In [26]:
# para o modelo down_sampled

X_train, X_test, y_train, y_test = train_test_split(train_downsampled['tweet'],                                    
                                                    train_downsampled['label'],random_state = 0)

model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)

# Atravé do f1 score vemos que ele é 85% preciso
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9694666666666667

# classificando

In [160]:
# pegando apenas o head do X_test
head_X_test = X_test.head()

head_X_test

head_X_test

1095                     bihday brother cool thebomb      
9766     gf asked me to make an account i told her i di...
26909         sunday   withyou happiness family  vinallop 
1386     been feeling low for ages and when the one per...
4616     chaplin  the dictator speech  via   theresista...
Name: tweet, dtype: object

In [161]:
ind = {}

cont = 0
for i in y_test.keys():
    ind[i] = cont 
    cont += 1
    
cont = 0
for i in head_X_test.keys():
    print(i)
    position_current = ind[i]
    if y_test[i] == y_predict[position_current]:
        print("Combinou")
        print(head_X_test[i])
        print("Valor de fato: %d"%(y_test[i]))
        print("Valor de previsto: %d"%(y_predict[position_current]))
        print("")
    else:
        print('Não combinou!!')
        print(head_X_test[i])
        print("Valor de fato: %d"%(y_test[i]))
        print("Valor de previsto: %d"%(y_predict[position_current]))
        print("")


1095
Combinou
  bihday brother cool thebomb      
Valor de fato: 0
Valor de previsto: 0

9766
Combinou
gf asked me to make an account i told her i did not want to we compromised and i made a twitter account    gf life
Valor de fato: 0
Valor de previsto: 0

26909
Combinou
sunday   withyou happiness family  vinallop 
Valor de fato: 0
Valor de previsto: 0

1386
Combinou
been feeling low for ages and when the one person you want to talk to isnt responding but you knows theyre online  feelingcrapmehhhhh
Valor de fato: 0
Valor de previsto: 0

4616


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().